In [52]:
!pip install textblob
import pandas as pd
from textblob import TextBlob
import matplotlib.pyplot as plt
import plotly.express as px

In [0]:
# # Read in all CSVs to dataframes
# df1 = pd.read_csv('https://raw.githubusercontent.com/serinamarie/data-science-1/master/results1.csv')
# df2 = pd.read_csv('https://raw.githubusercontent.com/serinamarie/data-science-1/master/results2.csv')
# df3 = pd.read_csv('https://raw.githubusercontent.com/serinamarie/data-science-1/master/results3.csv')
# df4 = pd.read_csv('https://raw.githubusercontent.com/serinamarie/data-science-1/master/results4.csv')
# df5 = pd.read_csv('https://media.githubusercontent.com/media/serinamarie/GIT-LFS-storage/master/results5.csv')

In [0]:
# Read in all CSVs to dataframes
df1 = pd.read_csv('results1.csv')
df2 = pd.read_csv('results2.csv')
df3 = pd.read_csv('results3.csv')
df4 = pd.read_csv('results4.csv')
df5 = pd.read_csv('results5.csv')

In [0]:
def clean(df1, df2, df3, df4, df5):

    ## Drop unnecessary columns and concatenate dataframes
    cols_to_drop1 = ['title','url','dead','score', 'time', 'type','id','parent','descendants','ranking','deleted']
    df1 = df1.drop(columns=cols_to_drop1)
    df1 = df1.rename(columns={'by':'user'})

    cols_to_drop2 = ['title','url','dead','score', 'time', 'timestamp','type','id','parent','descendants','ranking','deleted']
    df2 = df2.drop(columns=cols_to_drop2)
    df2 = df2.rename(columns={'by':'user'})
    df2 = df2[['user','text']].dropna()

    cols_to_drop3 = ['title','url','dead','score', 'time', 'timestamp','type','id','parent','descendants','ranking','deleted']
    df3 = df3.drop(columns=cols_to_drop3)
    df3 = df3.rename(columns={'by':'user'})
    df3 = df3[['user','text']].dropna()

    cols_to_drop4 = ['title','url','dead','score', 'time', 'type','id','parent','descendants','ranking','deleted']
    df4 = df4.drop(columns=cols_to_drop4)
    df4 = df4.rename(columns={'by':'user'})
    
    cols_to_drop5 = ['item_id']
    df5 = df5.drop(columns=cols_to_drop5)
    df5 = df5.rename(columns={'user_name':'user'})

    return df1, df2, df3, df4, df5
    # # Concatenate
    # pdList = [df1, df2, df3, df4, df5]  # List of dataframes
    # df = pd.concat(pdList)
    # return df

In [0]:
# Apply our function
df1, df2, df3, df4, df5 = clean(df1,df2,df3,df4, df5)

In [66]:
df1.shape, df2.shape, df3.shape, df4.shape, df5.shape

((16000, 2), (13250, 2), (13336, 2), (16000, 2), (634325, 2))

In [0]:
def wrangle(df):
    
    # Do some text processing (not perfect)
    def preprocess(row):
        
        row = str(row)
        row = row.replace('<p>','')
        row = row.replace('\\','')
        row = row.replace('&#62;','')
        row = row.replace('<pre>','')
        row = row.replace('<code>','')
        row = row.replace('</code>','')
        row = row.replace('</pre>','')
        row = row.replace('&quot;','')
        row = row.replace('&#x27;','')
        row = row.replace('<i>','')
        row = row.replace('</i>','')
        
        return row
    
    df['text'] = [preprocess(t) for t in df['text']]

    # Analyze sentiment and append in new columns
    df['blobbed'] = [TextBlob(t) for t in df['text']]
    df['polarity'] = [b.sentiment[0] for b in df['blobbed']]
    df['subjectivity'] = [b.sentiment[1] for b in df['blobbed']]

    # Identify subjective and negative comments
    df['salty'] = (df['subjectivity'] >= 0.5) & (df['polarity'] <= 0)

    # Identify subjective and positive comments
    df['sweet'] = (df['subjectivity'] >= 0.5) & (df['polarity'] >= 0)

    # Identify objective and negative comments
    df['astringent'] = (df['subjectivity'] <= 0.5) & (df['polarity'] <= 0)

    # Identify objective and positive comments
    df['savory'] = (df['subjectivity'] <= 0.5) & (df['polarity'] >= 0)   

    # Create a new column containing the tastiness category of each comment
    def taste_test(row):
        if row['savory'] == True:
            val = 'savory'
        elif row['sweet'] == True:
            val = 'sweet'
        elif row['salty'] == True:
            val = 'salty'
        elif row['astringent'] == True:
            val = 'astringent'
        return val

    # Apply our taste_test function
    df['taste_cat'] = df.apply(taste_test, axis=1)
    # Numeric version for visualization
    df['taste_num'] = df['taste_cat'].map({'savory':0,'sweet':1,'salty':2,'astringent':3})

    # We want our polar_sub score to not to be zeroed out during multiplication
    df['subjectivity'] = df['subjectivity'].replace(0.000000,0.000001)
    df['polarity'] = df['polarity'].replace(0.000000,0.000001)
    df['polar_sub'] = df['polarity'] * df['subjectivity']

    # Drop unnecessary columns 
    df = df.drop(columns=['blobbed'])

    # Rename appropriately
    df = df.rename(columns={'text': 'comment'})
    
    # dang is a moderator - dang patrols the trolls but is mucking up the leaderboard
    df = df[df['user'] != 'dang']
    
    df.drop_duplicates(keep=False,inplace=True) 
    
    # Links seem to mess up the polar_sub score
    df = df[~df['comment'].str.contains('<a href')]

    return df

In [69]:
# Apply function
df1 = wrangle(df1)
df1.head()

,user,comment,polarity,subjectivity,salty,sweet,astringent,savory,taste_cat,taste_num,polar_sub
0,danmaz74,Do those analyses also account for the energy ...,0.000001,0.000001,False,False,True,True,savory,0,1.000000e-12
2,qeorge,It seems that IE has been a consistent impedim...,0.345000,0.325000,False,False,False,True,savory,0,1.121250e-01
5,zxcvvcxz,Not a fan of this. I think the OP exhibits som...,0.147428,0.513579,False,True,False,False,sweet,1,7.571597e-02
6,eru,None. Interferes with my concentration.,0.000001,0.000001,False,False,True,True,savory,0,1.000000e-12
7,raphaelss,Location: Brazil.Remote: Yes.Willint to reloca...,0.100000,0.421429,False,False,False,True,savory,0,4.214286e-02


In [70]:
df2 = wrangle(df2)
df2.head()

,user,comment,polarity,subjectivity,salty,sweet,astringent,savory,taste_cat,taste_num,polar_sub
1,iamnewhere,Im currently at a company which lacks a produc...,0.108333,0.525000,False,True,False,False,sweet,1,5.687500e-02
6,adv0r,How do you deal with technical debt in startup...,0.000001,0.050000,False,False,True,True,savory,0,5.000000e-08
12,jacquesm,#NAME?,0.000001,0.000001,False,False,True,True,savory,0,1.000000e-12
28,ghotli,About a month ago quite a few of you identifie...,0.150000,0.300000,False,False,False,True,savory,0,4.500000e-02
40,solipsist,"Over the years, Y Combinator has seem to have ...",0.543333,0.808333,False,True,False,False,sweet,1,4.391944e-01


In [71]:
df3 = wrangle(df3)
df3.head()

,user,comment,polarity,subjectivity,salty,sweet,astringent,savory,taste_cat,taste_num,polar_sub
0,red75prime,Presbyopia. Humans eyes gradually lose ability...,-0.250000,0.250000,False,False,True,False,astringent,3,-6.250000e-02
1,dmix,How much of a fluctuation did you test? $9 vs ...,0.200000,0.200000,False,False,False,True,savory,0,4.000000e-02
3,corin_,Agreed that it was probably accidental rather ...,-0.400000,0.600000,True,False,False,False,salty,2,-2.400000e-01
4,lisper,There is no more possibility of an accident wi...,0.080000,0.515000,False,True,False,False,sweet,1,4.120000e-02
6,ddw,But emails though,0.000001,0.000001,False,False,True,True,savory,0,1.000000e-12


In [72]:
df4 = wrangle(df4)
df4.head()

,user,comment,polarity,subjectivity,salty,sweet,astringent,savory,taste_cat,taste_num,polar_sub
0,danmaz74,Do those analyses also account for the energy ...,0.000001,0.000001,False,False,True,True,savory,0,1.000000e-12
2,qeorge,It seems that IE has been a consistent impedim...,0.345000,0.325000,False,False,False,True,savory,0,1.121250e-01
5,zxcvvcxz,Not a fan of this. I think the OP exhibits som...,0.147428,0.513579,False,True,False,False,sweet,1,7.571597e-02
6,eru,None. Interferes with my concentration.,0.000001,0.000001,False,False,True,True,savory,0,1.000000e-12
7,raphaelss,Location: Brazil.Remote: Yes.Willint to reloca...,0.100000,0.421429,False,False,False,True,savory,0,4.214286e-02


In [73]:
df5 = wrangle(df5)
df5.head()

,user,comment,polarity,subjectivity,salty,sweet,astringent,savory,taste_cat,taste_num,polar_sub
0,makomk,Its implemented using a few standard 74xxx TTL...,0.060000,0.320000,False,False,False,True,savory,0,1.920000e-02
1,gopowerranger,Why is this even a question asked? Its child-l...,0.000001,0.000001,False,False,True,True,savory,0,1.000000e-12
2,rayiner,Is taxing expats abroad really a problem? Firs...,0.103274,0.425992,False,False,False,True,savory,0,4.399382e-02
3,mikeash,"They do use many US public services, though. A...",0.093095,0.289643,False,False,False,True,savory,0,2.696437e-02
4,zensavona,"Not sure where in Europe you mean, but I live ...",-0.065227,0.715278,True,False,False,False,salty,2,-4.665562e-02


In [0]:
# Concatenate
pdList = [df1, df2, df3, df4, df5]  # List of dataframes
df = pd.concat(pdList)

In [0]:
# Done with graphs
def done_with_graphs(df):
    df = df.drop(columns=['polarity', 'subjectivity','taste_num'])
    return df

df = done_with_graphs(df)

In [77]:
# View the engineered features
df.head()

,user,comment,salty,sweet,astringent,savory,taste_cat,polar_sub
0,danmaz74,Do those analyses also account for the energy ...,False,False,True,True,savory,1.000000e-12
2,qeorge,It seems that IE has been a consistent impedim...,False,False,False,True,savory,1.121250e-01
5,zxcvvcxz,Not a fan of this. I think the OP exhibits som...,False,True,False,False,sweet,7.571597e-02
6,eru,None. Interferes with my concentration.,False,False,True,True,savory,1.000000e-12
7,raphaelss,Location: Brazil.Remote: Yes.Willint to reloca...,False,False,False,True,savory,4.214286e-02


In [78]:
# Full dataframe for database
df_for_db = df.drop(columns=['salty','sweet','astringent','savory','taste_cat']).rename(columns={'polar_sub':'salt_score'})
df_for_db.head()
# df_for_db.to_json(r'df.json', orient='records')

,user,comment,salt_score
0,danmaz74,Do those analyses also account for the energy ...,1.000000e-12
2,qeorge,It seems that IE has been a consistent impedim...,1.121250e-01
5,zxcvvcxz,Not a fan of this. I think the OP exhibits som...,7.571597e-02
6,eru,None. Interferes with my concentration.,1.000000e-12
7,raphaelss,Location: Brazil.Remote: Yes.Willint to reloca...,4.214286e-02


In [79]:
# For .py file, write a function that returns a user and their comments
# Takes in a string as a parameter
def return_a_user(user):
    return df_for_db[df_for_db['user'] == user].sort_values(by='salt_score')
 
# Test it out 
return_a_user('pg').head()

,user,comment,salt_score
6906,pg,I can't wait. It's wrong that after all these...,-0.450000
6906,pg,I can't wait. It's wrong that after all these...,-0.450000
7006,pg,I suspect the biggest problem with Lisp adopti...,-0.178889
7042,pg,No. You don't get any richer by a trade at ma...,-0.062500
2214,pg,You underestimate the military-industrial comp...,-0.053333


In [0]:
'''SALTIEST USERS: 
Create a dataframe of the overall saltiest users alongside their saltiest comment'''

def salty_wrangler(df):

    # Get the highest salt scores ordered by user
    saltiest_users = df.groupby('user')['polar_sub'].sum().sort_values()[:100].index.to_list()

    # Create a new df to append all the saltiest users' saltiest comments
    df_saltiest_comment = pd.DataFrame()

    # Iterate through each user, getting their saltiest comment
    for user in saltiest_users:
        df_saltiest_comment = df_saltiest_comment.append(df[(df['user'] == user) & (df['taste_cat'] == 'salty')].sort_values(by='polar_sub', ascending=True).head(1))

    ## Create our new dataframe 

    # Create a df of their salt score
    new = pd.DataFrame(df.groupby('user')['polar_sub'].sum()).reset_index()

    # Merge the new df with our users' saltiest comments
    new = new.merge(df_saltiest_comment[['comment', 'user']], on='user')
    new = new.rename(columns={'comment':'saltiest_comment', 'polar_sub':'salt_score'})

    # Rank our saltiest users from highest salt score to lowest
    new['rank'] = new['salt_score'].rank(ascending=True, method='first').astype(int)
    new.sort_values('rank', inplace=True) 
   
    # Order our columns as we like
    new = new[['rank', 'user', 'salt_score', 'saltiest_comment']]

    return new

In [0]:
saltiest_hackers = salty_wrangler(df)

In [0]:
# SWEETEST HACKERS

def sweet_wrangler(df):

    # Get the highest sugar scores ordered by user
    sweetest_users = df.groupby('user')['polar_sub'].sum().sort_values(ascending=False)[:100].index.to_list()

    # Create a new df to append all the sweetest users' sweetest comments
    df_sweetest_comment = pd.DataFrame()
    # Iterate through each user, getting their sweetest comment
    for user in sweetest_users:
        df_sweetest_comment = df_sweetest_comment.append(df[(df['user'] == user) & (df['taste_cat'] == 'sweet')].sort_values(by='polar_sub', ascending=False).head(1))

    ## Create our new dataframe 

    # Create a df of their sugar score
    new = pd.DataFrame(df.groupby('user')['polar_sub'].sum()).reset_index()

    # Merge the new df with our users' sweetest comments
    new = new.merge(df_sweetest_comment[['comment', 'user']], on='user')
    new = new.rename(columns={'comment':'sweetest_comment', 'polar_sub':'sugar_score'})

    # rank our saltiest users from highest salt score to lowest
    new['rank'] = new['sugar_score'].rank(ascending=False, method='first').astype(int)
    new.sort_values('rank', inplace=True) 

    # Order our columns as we like
    new = new[['rank', 'user', 'sugar_score', 'sweetest_comment']]

    return new

In [0]:
sweetest_hackers = sweet_wrangler(df)

In [0]:
# Bye, bye, data
sweet = sweetest_hackers.to_json(r'sweet.json', orient='records')
salty = saltiest_hackers.to_json(r'salty.json', orient='records')
df = df.to_json(r'df.json', orient='records')